Install required modules

In [ ]:
%pip install kagglehub 
%pip install numpy pandas matplotlib seaborn scikit-learn scikit-image tqdm

Load Modules

In [2]:
# Load Modules
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

Download the dataset and save it localy

In [ ]:
import os
import kagglehub
import shutil
import tqdm

# Create data directory if it doesn't exist
os.makedirs("./data", exist_ok=True)

# Download the dataset (this will go to a temporary location)
path = kagglehub.dataset_download("pavansubhasht/ibm-hr-analytics-attrition-dataset")

# Move the downloaded file to your desired location
path = path+"/WA_Fn-UseC_-HR-Employee-Attrition.csv"
shutil.copy(path, "./data/ibm_hr_dataset.csv")

Load the dataset

In [ ]:
"""Load and prepare the IBM HR Analytics dataset"""
df = pd.read_csv('./data/ibm_hr_dataset.csv')

# Convert categorical variables to numeric
categorical_columns = ['BusinessTravel', 'Department', 'EducationField', 
                        'Gender', 'JobRole', 'MaritalStatus', 'OverTime']
df_encoded = pd.get_dummies(df, columns=categorical_columns)

df, df_encoded

Basic overview of company wide attrition, yes have left the company, while no have not left the company.

In [ ]:
# Cell 1: Basic Attrition Distribution
# Visualize overall attrition distribution
attrition_dist = df['Attrition'].value_counts()
attrition_pct = df['Attrition'].value_counts(normalize=True) * 100

plt.figure(figsize=(10, 6))
sns.countplot(data=df, x='Attrition')
plt.title('Employee Attrition Distribution')
plt.show()

print("\nAttrition Distribution:")
for label, count, percentage in zip(attrition_dist.index, attrition_dist.values, attrition_pct.values):
    print(f"{label}: {count} employees ({percentage:.1f}%)")

In this series of box we see attrition rate for sevreal features.  Yes employees are leaving the company, and no employees are staying.

In [ ]:
# Cell 2: Numerical Features Analysis
# Analyze numerical features vs attrition
numerical_features = ['Age', 'DailyRate', 'DistanceFromHome', 'MonthlyIncome', 
                     'NumCompaniesWorked', 'PercentSalaryHike', 'TotalWorkingYears', 
                     'YearsAtCompany', 'YearsInCurrentRole']

plt.figure(figsize=(15, 10))
for i, feature in enumerate(numerical_features, 1):
    plt.subplot(3, 3, i)
    sns.boxplot(data=df, x='Attrition', y=feature)
    plt.title(f'{feature} vs Attrition')
plt.tight_layout()
plt.show()

Here we look at the attrition rate for each category of the categorical features.  We can see that employees who have left the company are more likely to have a higher distance from home, in Sales, and are single with a lower monthly income and education and work more overtime.

In [ ]:
# Cell 3: Categorical Features Analysis
# Analyze categorical features vs attrition
categorical_features = ['BusinessTravel', 'Department', 'Education', 
                       'JobRole', 'MaritalStatus', 'OverTime']

plt.figure(figsize=(15, 10))
for i, feature in enumerate(categorical_features, 1):
    plt.subplot(2, 3, i)
    df_pct = df.groupby(feature)['Attrition'].value_counts(normalize=True).unstack()
    df_pct['Yes'].sort_values(ascending=False).plot(kind='bar')
    plt.title(f'Attrition Rate by {feature}')
    plt.xlabel(feature)
    plt.ylabel('Attrition Rate')
    plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

Here we handle our pre processing.  We convert our categorical variables to numeric using one hot encoding.  We also convert our target variable to numeric.  We then split our data into features and target.

In [8]:
# Cell 4: Data Preprocessing
# Convert categorical variables to numeric
categorical_columns = ['BusinessTravel', 'Department', 'EducationField', 
                      'Gender', 'JobRole', 'MaritalStatus', 'OverTime', 'Over18']
df_encoded = pd.get_dummies(df, columns=categorical_columns)

# Convert 'Attrition' to numeric
df_encoded['Attrition'] = (df['Attrition'] == 'Yes').astype(int)

# Split features and target
X = df_encoded.drop('Attrition', axis=1)
y = df_encoded['Attrition']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Now that we have pre processed our data we can train our model.  We use a random forest classifier.  We then make predictions and print the classification report.

In [ ]:
# Cell 5: Model Training and Evaluation
# Train Random Forest model
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train_scaled, y_train)

# Make predictions
y_pred = rf_model.predict(X_test_scaled)

# Print classification report
print("Model Performance:")
print(classification_report(y_test, y_pred))

Here we look at the importance of each feature in predicting attrition.  We can see that the top features are the monthly income, age and overtime.

In [ ]:
# Cell 6: Feature Importance Visualization
# Get feature importance
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': rf_model.feature_importances_
})

# Plot top 15 most important features
plt.figure(figsize=(12, 6))
feature_importance.sort_values('importance', ascending=False).head(15).plot(
    x='feature', y='importance', kind='bar')
plt.title('Top 15 Most Important Features for Predicting Attrition')
plt.xlabel('Features')
plt.ylabel('Importance')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

Here we determine what a high attrition employee is.  We can see that they are older, have a higher monthly income, have worked at more companies, and have a higher percent salary hike.


In [ ]:
# Profile of High Attrition Employees
# Calculate mean values for numerical features by attrition status
numerical_features = ['Age', 'DailyRate', 'DistanceFromHome', 'MonthlyIncome', 
                     'NumCompaniesWorked', 'PercentSalaryHike', 'TotalWorkingYears', 
                     'YearsAtCompany', 'YearsInCurrentRole']

profile = df.groupby('Attrition')[numerical_features].mean()

# Calculate percentages for categorical features
categorical_features = ['BusinessTravel', 'Department', 'Education', 
                       'JobRole', 'MaritalStatus', 'OverTime']

cat_profile = {}
for feature in categorical_features:
    cat_profile[feature] = df[df['Attrition'] == 'Yes'][feature].value_counts(normalize=True)

# Print the profile
print("Profile of Employees Who Leave (High Attrition):")
print("\nNumerical Characteristics (Averages):")
print(profile.loc['Yes'].round(2))

print("\nCategorical Characteristics (Top Categories):")
for feature, dist in cat_profile.items():
    print(f"\n{feature}:")
    print(dist.head(3).round(3) * 100)

Finally we look at the full dataset and determine the 5 most likely to leave the company.  We can see that they are older, have a higher monthly income, have worked at more companies, and have a higher percent salary hike and what it would take to keep them.

In [ ]:
# Use our trained model to get probability predictions for all employees
X_all_scaled = scaler.transform(X)
attrition_probs = rf_model.predict_proba(X_all_scaled)[:, 1]  # Probability of leaving

# Add probabilities to original dataframe
df['Attrition_Risk'] = attrition_probs

# Get top 5 at-risk employees with their key characteristics
high_risk = df.nlargest(5, 'Attrition_Risk')[['Age', 'JobRole', 'MonthlyIncome', 
                                             'OverTime', 'DistanceFromHome', 
                                             'YearsAtCompany', 'JobSatisfaction',
                                             'WorkLifeBalance', 'Department',
                                             'Attrition_Risk']]

# Print detailed analysis for each high-risk employee
print("Top 5 Employees at Risk of Leaving:\n")
for idx, employee in high_risk.iterrows():
    print(f"Employee Risk Score: {employee['Attrition_Risk']:.2%}")
    print(f"Current Role: {employee['JobRole']}")
    print(f"Department: {employee['Department']}")
    print(f"Age: {employee['Age']}")
    print(f"Monthly Income: ${employee['MonthlyIncome']:,.2f}")
    print(f"Works Overtime: {employee['OverTime']}")
    print(f"Distance From Home: {employee['DistanceFromHome']} miles")
    print(f"Years at Company: {employee['YearsAtCompany']}")
    print(f"Job Satisfaction: {employee['JobSatisfaction']}/4")
    print(f"Work Life Balance: {employee['WorkLifeBalance']}/4")
    
    # Recommend retention strategies based on key factors
    print("\nRetention Recommendations:")
    recommendations = []
    
    if employee['OverTime'] == 'Yes':
        recommendations.append("- Consider reducing overtime or providing additional compensation/time off")
    
    if employee['JobSatisfaction'] < 3:
        recommendations.append("- Schedule career development discussion and explore growth opportunities")
    
    if employee['MonthlyIncome'] < df[df['JobRole'] == employee['JobRole']]['MonthlyIncome'].mean():
        recommendations.append("- Review compensation relative to role peers")
    
    if employee['WorkLifeBalance'] < 3:
        recommendations.append("- Evaluate workload and consider flexible working arrangements")
    
    if employee['DistanceFromHome'] > 10:
        recommendations.append("- Discuss hybrid/remote work options")
    
    for rec in recommendations:
        print(rec)
    print("\n" + "-"*50 + "\n")